# 전처리

In [38]:
import pandas as pd
import warnings 
import numpy as np

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import StandardScaler

### 파일 불러오기

In [39]:
data = pd.read_excel("data/data_weather.xlsx")
data.tail()

,타임스탬프,귀하의 성별은 무엇입니까?,귀하의 연령대는 무엇입니까?,귀하의 거주 지역은 어디입니까?,오늘 기분은 어떠십니까?,1순위 음식 분야,2순위 음식 분야,3순위 음식 분야,1_한식,2_한식,...,2_일식,3_일식,1_중식,2_중식,3_중식,기온(°C),강수량(mm),습도(%),현지기압(hPa),전운량(10분위)
438,2021-05-24-16,남자,노년 (만 65세 이상),경상도,행복하다 · 즐겁다 · 신난다,한식,분식,중식,김치찌개,NaN,...,NaN,NaN,NaN,NaN,짬뽕,23.8,0.0,52.2,994.6,3
439,2021-05-24-16,남자,노년 (만 65세 이상),"수도권 (경기, 인천)",행복하다 · 즐겁다 · 신난다,한식,일식,중식,비빔밥,NaN,...,메밀소바,NaN,NaN,NaN,짜장면,22.1,0.0,45.1,999.9,1
440,2021-05-24-16,여자,노년 (만 65세 이상),전라도,행복하다 · 즐겁다 · 신난다,한식,중식,일식,비빔밥,NaN,...,NaN,돈가스,NaN,볶음밥,NaN,22.7,0.0,57.0,997.9,1
441,2021-05-24-16,남자,노년 (만 65세 이상),전라도,우울하다 · 피곤하다 · 힘들다,분식,양식,일식,NaN,NaN,...,NaN,메밀소바,NaN,NaN,NaN,22.7,0.0,57.0,997.9,1
442,2021-05-24-16,여자,노년 (만 65세 이상),전라도,행복하다 · 즐겁다 · 신난다,한식,중식,일식,냉면,NaN,...,NaN,메밀소바,NaN,짜장면,NaN,22.7,0.0,57.0,997.9,1


In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   타임스탬프              443 non-null    object 
 1   귀하의 성별은 무엇입니까?     443 non-null    object 
 2   귀하의 연령대는 무엇입니까?    443 non-null    object 
 3   귀하의 거주 지역은 어디입니까?  443 non-null    object 
 4   오늘 기분은 어떠십니까?      443 non-null    object 
 5   1순위 음식 분야          443 non-null    object 
 6   2순위 음식 분야          443 non-null    object 
 7   3순위 음식 분야          443 non-null    object 
 8   1_한식               208 non-null    object 
 9   2_한식               110 non-null    object 
 10  3_한식               77 non-null     object 
 11  1_분식               40 non-null     object 
 12  2_분식               44 non-null     object 
 13  3_분식               71 non-null     object 
 14  1_양식               81 non-null     object 
 15  2_양식               80 non-null     object 
 16  3_양식               98 non-

## 컬럼명 정리

### 컬럼명 변경

In [41]:
data.columns

Index(['타임스탬프', '귀하의 성별은 무엇입니까?', '귀하의 연령대는 무엇입니까?', '귀하의 거주 지역은 어디입니까?',
       '오늘 기분은 어떠십니까?', '1순위 음식 분야', '2순위 음식 분야', '3순위 음식 분야', '1_한식', '2_한식',
       '3_한식', '1_분식', '2_분식', '3_분식', '1_양식', '2_양식', '3_양식', '1_일식', '2_일식',
       '3_일식', '1_중식', '2_중식', '3_중식', '기온(°C)', '강수량(mm)', '습도(%)',
       '현지기압(hPa)', '전운량(10분위)'],
      dtype='object')

In [42]:
# rename
data.rename(columns={'타임스탬프':'시간대',
                     '귀하의 성별은 무엇입니까?':'성별',
                     '귀하의 연령대는 무엇입니까?':'연령대',
                     '귀하의 거주 지역은 어디입니까?':'지역',
                     '오늘 기분은 어떠십니까?':'기분',
                     '1순위 음식 분야':'1순위',
                     '2순위 음식 분야':'2순위',
                     '3순위 음식 분야':'3순위',
                     '기온(°C)':'기온',
                     '강수량(mm)':'강수량',
                     '습도(%)':'습도',
                     '현지기압(hPa)':'기압',
                     '전운량(10분위)':'전운량'}, inplace=True)

In [43]:
data.head()

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,1_한식,2_한식,...,2_일식,3_일식,1_중식,2_중식,3_중식,기온,강수량,습도,기압,전운량
0,2021-05-16-14,남자,청년 (만 19세~만 34세 이하),"수도권 (경기, 인천)",행복하다 · 즐겁다 · 신난다,한식,일식,중식,"불고기, 김치찌개",NaN,...,"돈가스, 초밥",NaN,NaN,NaN,짬뽕,17.5,1.6,99.1,993.6,10
1,2021-05-16-15,여자,중장년 (만 35세~만 64세 이하),서울,화가 난다 · 짜증난다,한식,중식,일식,불고기,NaN,...,NaN,메밀소바,NaN,볶음밥,NaN,18.5,1.4,97.0,989.6,10
2,2021-05-16-15,남자,청년 (만 19세~만 34세 이하),서울,아무 생각 없다,일식,양식,한식,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
3,2021-05-16-15,여자,청년 (만 19세~만 34세 이하),서울,행복하다 · 즐겁다 · 신난다,일식,한식,분식,NaN,비빔밥,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
4,2021-05-16-16,남자,청년 (만 19세~만 34세 이하),경상도,행복하다 · 즐겁다 · 신난다,양식,일식,분식,NaN,NaN,...,"돈가스, 초밥",NaN,NaN,NaN,NaN,20.6,2.6,94.7,990.5,10


In [44]:
data

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,1_한식,2_한식,...,2_일식,3_일식,1_중식,2_중식,3_중식,기온,강수량,습도,기압,전운량
0,2021-05-16-14,남자,청년 (만 19세~만 34세 이하),"수도권 (경기, 인천)",행복하다 · 즐겁다 · 신난다,한식,일식,중식,"불고기, 김치찌개",NaN,...,"돈가스, 초밥",NaN,NaN,NaN,짬뽕,17.5,1.6,99.1,993.6,10
1,2021-05-16-15,여자,중장년 (만 35세~만 64세 이하),서울,화가 난다 · 짜증난다,한식,중식,일식,불고기,NaN,...,NaN,메밀소바,NaN,볶음밥,NaN,18.5,1.4,97.0,989.6,10
2,2021-05-16-15,남자,청년 (만 19세~만 34세 이하),서울,아무 생각 없다,일식,양식,한식,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
3,2021-05-16-15,여자,청년 (만 19세~만 34세 이하),서울,행복하다 · 즐겁다 · 신난다,일식,한식,분식,NaN,비빔밥,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
4,2021-05-16-16,남자,청년 (만 19세~만 34세 이하),경상도,행복하다 · 즐겁다 · 신난다,양식,일식,분식,NaN,NaN,...,"돈가스, 초밥",NaN,NaN,NaN,NaN,20.6,2.6,94.7,990.5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,2021-05-24-16,남자,노년 (만 65세 이상),경상도,행복하다 · 즐겁다 · 신난다,한식,분식,중식,김치찌개,NaN,...,NaN,NaN,NaN,NaN,짬뽕,23.8,0.0,52.2,994.6,3
439,2021-05-24-16,남자,노년 (만 65세 이상),"수도권 (경기, 인천)",행복하다 · 즐겁다 · 신난다,한식,일식,중식,비빔밥,NaN,...,메밀소바,NaN,NaN,NaN,짜장면,22.1,0.0,45.1,999.9,1
440,2021-05-24-16,여자,노년 (만 65세 이상),전라도,행복하다 · 즐겁다 · 신난다,한식,중식,일식,비빔밥,NaN,...,NaN,돈가스,NaN,볶음밥,NaN,22.7,0.0,57.0,997.9,1
441,2021-05-24-16,남자,노년 (만 65세 이상),전라도,우울하다 · 피곤하다 · 힘들다,분식,양식,일식,NaN,NaN,...,NaN,메밀소바,NaN,NaN,NaN,22.7,0.0,57.0,997.9,1


##   데이터 중복값,결측치 처리

In [45]:
# 결측치 확인
data.isnull().sum()

시간대       0
성별        0
연령대       0
지역        0
기분        0
1순위       0
2순위       0
3순위       0
1_한식    235
2_한식    333
3_한식    366
1_분식    403
2_분식    399
3_분식    372
1_양식    362
2_양식    363
3_양식    345
1_일식    356
2_일식    317
3_일식    333
1_중식    416
2_중식    360
3_중식    356
기온        0
강수량       0
습도        0
기압        0
전운량       0
dtype: int64

### 1순위 ,2순위,3순위 겹치는 열 제거

In [46]:
data[(data["1순위"]==data["2순위"]) | (data["1순위"]==data["3순위"]) | (data["1순위"]==data["3순위"])]

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,1_한식,2_한식,...,2_일식,3_일식,1_중식,2_중식,3_중식,기온,강수량,습도,기압,전운량
86,2021-05-16-17,남자,청년 (만 19세~만 34세 이하),서울,아무 생각 없다,한식,한식,한식,된장찌개,된장찌개,...,NaN,NaN,NaN,NaN,NaN,17.4,4.3,98.0,989.2,10
88,2021-05-16-17,남자,청년 (만 19세~만 34세 이하),충청도,우울하다 · 피곤하다 · 힘들다,한식,양식,한식,불고기,NaN,...,NaN,NaN,NaN,NaN,NaN,20.1,3.2,96.0,987.1,10
95,2021-05-16-18,남자,청년 (만 19세~만 34세 이하),경상도,아무 생각 없다,분식,분식,중식,NaN,NaN,...,NaN,NaN,NaN,NaN,짜장면,19.7,2.7,94.9,990.0,9
96,2021-05-16-18,남자,청소년 (~만 19세 미만),경상도,행복하다 · 즐겁다 · 신난다,한식,한식,한식,냉면,김치찌개,...,NaN,NaN,NaN,NaN,NaN,19.7,2.7,94.9,990.0,9
100,2021-05-16-18,남자,청소년 (~만 19세 미만),경상도,아무 생각 없다,양식,양식,양식,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,19.7,2.7,94.9,990.0,9
133,2021-05-16-21,남자,청소년 (~만 19세 미만),경상도,아무 생각 없다,일식,일식,일식,NaN,NaN,...,돈가스,돈가스,NaN,NaN,NaN,18.1,1.6,95.4,991.6,9
154,2021-05-17-09,남자,청년 (만 19세~만 34세 이하),서울,우울하다 · 피곤하다 · 힘들다,한식,한식,일식,불고기,비빔밥,...,NaN,돈가스,NaN,NaN,NaN,14.9,0.0,94.0,996.0,10
161,2021-05-17-10,여자,청년 (만 19세~만 34세 이하),서울,우울하다 · 피곤하다 · 힘들다,한식,한식,일식,김치찌개,된장찌개,...,NaN,초밥,NaN,NaN,NaN,15.4,0.0,92.0,996.4,10
162,2021-05-17-10,남자,청년 (만 19세~만 34세 이하),서울,아무 생각 없다,일식,한식,일식,NaN,냉면,...,NaN,초밥,NaN,NaN,NaN,15.4,0.0,92.0,996.4,10
189,2021-05-17-18,여자,중장년 (만 35세~만 64세 이하),경상도,우울하다 · 피곤하다 · 힘들다,분식,분식,일식,NaN,NaN,...,NaN,우동,NaN,NaN,NaN,15.6,0.5,95.3,997.3,9


In [47]:
data=data[~((data["1순위"]==data["2순위"]) | (data["1순위"]==data["3순위"]) | (data["1순위"]==data["3순위"]))]

In [48]:
data.reset_index(drop=True,inplace=True)

### 입력 데이터 값 전처리

In [49]:
data.loc[(data['지역']=='수도권 (경기, 인천)'), '지역'] = '수도권'
data.loc[(data['연령대']=='청소년 (~만 19세 미만)'), '연령대'] = '청소년'
data.loc[(data['연령대']=='청년 (만 19세~만 34세 이하)'), '연령대'] = '청년'
data.loc[(data['연령대']=='중장년 (만 35세~만 64세 이하)'), '연령대'] = '중장년'
data.loc[(data['연령대']=='노년 (만 65세 이상)'), '연령대'] = '노년'

C:\Users\JUNG MIN GUE\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [50]:
data.loc[(data['기분']=='행복하다 · 즐겁다 · 신난다'), '기분'] = '행복'
data.loc[(data['기분']=='화가 난다 · 짜증난다'), '기분'] = '분노'
data.loc[(data['기분']=='아무 생각 없다'), '기분'] = '중립'
data.loc[(data['기분']=='우울하다 · 피곤하다 · 힘들다'), '기분'] = '우울'

In [51]:
data

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,1_한식,2_한식,...,2_일식,3_일식,1_중식,2_중식,3_중식,기온,강수량,습도,기압,전운량
0,2021-05-16-14,남자,청년,수도권,행복,한식,일식,중식,"불고기, 김치찌개",NaN,...,"돈가스, 초밥",NaN,NaN,NaN,짬뽕,17.5,1.6,99.1,993.6,10
1,2021-05-16-15,여자,중장년,서울,분노,한식,중식,일식,불고기,NaN,...,NaN,메밀소바,NaN,볶음밥,NaN,18.5,1.4,97.0,989.6,10
2,2021-05-16-15,남자,청년,서울,중립,일식,양식,한식,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
3,2021-05-16-15,여자,청년,서울,행복,일식,한식,분식,NaN,비빔밥,...,NaN,NaN,NaN,NaN,NaN,18.5,1.4,97.0,989.6,10
4,2021-05-16-16,남자,청년,경상도,행복,양식,일식,분식,NaN,NaN,...,"돈가스, 초밥",NaN,NaN,NaN,NaN,20.6,2.6,94.7,990.5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2021-05-24-16,남자,노년,경상도,행복,한식,분식,중식,김치찌개,NaN,...,NaN,NaN,NaN,NaN,짬뽕,23.8,0.0,52.2,994.6,3
417,2021-05-24-16,남자,노년,수도권,행복,한식,일식,중식,비빔밥,NaN,...,메밀소바,NaN,NaN,NaN,짜장면,22.1,0.0,45.1,999.9,1
418,2021-05-24-16,여자,노년,전라도,행복,한식,중식,일식,비빔밥,NaN,...,NaN,돈가스,NaN,볶음밥,NaN,22.7,0.0,57.0,997.9,1
419,2021-05-24-16,남자,노년,전라도,우울,분식,양식,일식,NaN,NaN,...,NaN,메밀소바,NaN,NaN,NaN,22.7,0.0,57.0,997.9,1


## 음식 피쳐 더미화 (One-Hot Encoding)

In [52]:
# 사용자가 선택한 메뉴만 뽑아내어 split 하는 Code

menu_data = data.iloc[:, 8:23]

for i in range(len(menu_data)):
    menu_data.iloc[i] = menu_data.iloc[i].str.split(', ')
    
menu_data

,1_한식,2_한식,3_한식,1_분식,2_분식,3_분식,1_양식,2_양식,3_양식,1_일식,2_일식,3_일식,1_중식,2_중식,3_중식
0,"[불고기, 김치찌개]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[돈가스, 초밥]",NaN,NaN,NaN,[짬뽕]
1,[불고기],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[메밀소바],NaN,[볶음밥],NaN
2,NaN,NaN,"[불고기, 김치찌개]",NaN,NaN,NaN,NaN,[스테이크],NaN,"[돈가스, 메밀소바]",NaN,NaN,NaN,NaN,NaN
3,NaN,[비빔밥],NaN,NaN,NaN,[떡볶이],NaN,NaN,NaN,[초밥],NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,[떡볶이],[햄버거],NaN,NaN,NaN,"[돈가스, 초밥]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,[김치찌개],NaN,NaN,NaN,[김밥],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[짬뽕]
417,[비빔밥],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[메밀소바],NaN,NaN,NaN,[짜장면]
418,[비빔밥],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[돈가스],NaN,[볶음밥],NaN
419,NaN,NaN,NaN,[떡볶이],NaN,NaN,NaN,[햄버거],NaN,NaN,NaN,[메밀소바],NaN,NaN,NaN


In [53]:
# 각 행의 메뉴를 한 리스트로 합치고, 이를 각 행 번호를 key로 갖는 딕셔너리로 만드는 Code

tmp_list = []
menu_dic = {}

for i in range(len(menu_data)):
    temp = menu_data.loc[i]
    for j in range(len(temp)):
        
        if type(temp.iloc[j]) == list:
            
            tmp_list += temp.iloc[j]
    
    menu_dic[i] = tmp_list
    tmp_list = []

In [54]:
# 각 카테고리에 해당하는 메뉴는 처음에 0으로 만들고,
## 사용자가 선택한 데이터를 1로 만드는 Code

category_dic = {'한식': ['불고기', '냉면', '칼국수', '된장찌개', '비빔밥', '김치찌개'],
          '분식': ['떡볶이', '김밥'],
          '양식': ['파스타', '스테이크', '햄버거'],
          '일식': ['초밥', '우동', '메밀소바', '돈가스'],
          '중식': ['볶음밥', '짬뽕', '짜장면']}

menu_list = ['불고기', '냉면', '칼국수', '된장찌개', '비빔밥', '김치찌개', '떡볶이', '김밥', '파스타',
'스테이크', '햄버거', '초밥', '우동', '메밀소바', '돈가스', '볶음밥', '짜장면', '짬뽕']

result_df = pd.DataFrame(columns=menu_list)

for i in range(len(data)):
    result_df.loc[i, category_dic[data.loc[i, '1순위']]] = 0 
    result_df.loc[i, category_dic[data.loc[i, '2순위']]] = 0 
    result_df.loc[i, category_dic[data.loc[i, '3순위']]] = 0 
    

    
    result_df.loc[i, menu_dic[i]] = 1
    
#     ##가중치 적용 
#     result_df.loc[i, category_dic[data.loc[i, '1순위']]]=result_df.loc[i, category_dic[data.loc[i, '1순위']]]*3
#     result_df.loc[i, category_dic[data.loc[i, '2순위']]]=result_df.loc[i, category_dic[data.loc[i, '2순위']]]*2
#     result_df.loc[i, category_dic[data.loc[i, '3순위']]]=result_df.loc[i, category_dic[data.loc[i, '3순위']]]*1
     
result_df=result_df.fillna(0)
result_df

,불고기,냉면,칼국수,된장찌개,비빔밥,김치찌개,떡볶이,김밥,파스타,스테이크,햄버거,초밥,우동,메밀소바,돈가스,볶음밥,짜장면,짬뽕
0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
417,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
418,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0
419,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0


In [55]:
result_df

,불고기,냉면,칼국수,된장찌개,비빔밥,김치찌개,떡볶이,김밥,파스타,스테이크,햄버거,초밥,우동,메밀소바,돈가스,볶음밥,짜장면,짬뽕
0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
2,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
417,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
418,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0
419,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0


## 최종 dataset 만들기

In [56]:
info = data.iloc[:, :8]
info

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위
0,2021-05-16-14,남자,청년,수도권,행복,한식,일식,중식
1,2021-05-16-15,여자,중장년,서울,분노,한식,중식,일식
2,2021-05-16-15,남자,청년,서울,중립,일식,양식,한식
3,2021-05-16-15,여자,청년,서울,행복,일식,한식,분식
4,2021-05-16-16,남자,청년,경상도,행복,양식,일식,분식
...,...,...,...,...,...,...,...,...
416,2021-05-24-16,남자,노년,경상도,행복,한식,분식,중식
417,2021-05-24-16,남자,노년,수도권,행복,한식,일식,중식
418,2021-05-24-16,여자,노년,전라도,행복,한식,중식,일식
419,2021-05-24-16,남자,노년,전라도,우울,분식,양식,일식


In [57]:
weather = data.iloc[:, -5:]
weather

,기온,강수량,습도,기압,전운량
0,17.5,1.6,99.1,993.6,10
1,18.5,1.4,97.0,989.6,10
2,18.5,1.4,97.0,989.6,10
3,18.5,1.4,97.0,989.6,10
4,20.6,2.6,94.7,990.5,10
...,...,...,...,...,...
416,23.8,0.0,52.2,994.6,3
417,22.1,0.0,45.1,999.9,1
418,22.7,0.0,57.0,997.9,1
419,22.7,0.0,57.0,997.9,1


## 파생변수 생성

### 필요한 컬럼만 concat

In [58]:
d1 = pd.concat([info, weather], axis=1)

In [59]:
final_data = pd.concat([d1, result_df], axis=1)

In [60]:
final_data['시간대']

0      2021-05-16-14
1      2021-05-16-15
2      2021-05-16-15
3      2021-05-16-15
4      2021-05-16-16
           ...      
416    2021-05-24-16
417    2021-05-24-16
418    2021-05-24-16
419    2021-05-24-16
420    2021-05-24-16
Name: 시간대, Length: 421, dtype: object

In [61]:
# 시간대를 datetime 타입으로 변경

final_data['시간대'] = pd.to_datetime(final_data['시간대'])

In [62]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   시간대     421 non-null    datetime64[ns]
 1   성별      421 non-null    object        
 2   연령대     421 non-null    object        
 3   지역      421 non-null    object        
 4   기분      421 non-null    object        
 5   1순위     421 non-null    object        
 6   2순위     421 non-null    object        
 7   3순위     421 non-null    object        
 8   기온      421 non-null    float64       
 9   강수량     421 non-null    float64       
 10  습도      421 non-null    float64       
 11  기압      421 non-null    float64       
 12  전운량     421 non-null    int64         
 13  불고기     421 non-null    int64         
 14  냉면      421 non-null    int64         
 15  칼국수     421 non-null    int64         
 16  된장찌개    421 non-null    int64         
 17  비빔밥     421 non-null    int64         
 18  김치찌개    42

### 날짜, 시간, 요일 컬럼 추가

In [63]:
final_data['날짜'] = final_data['시간대'].dt.strftime('%m-%d')
final_data['시간'] = final_data['시간대'].dt.strftime('%H')
final_data['요일'] = final_data['시간대'].dt.dayofweek

In [64]:
final_data

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,기온,강수량,...,초밥,우동,메밀소바,돈가스,볶음밥,짜장면,짬뽕,날짜,시간,요일
0,2021-05-16 14:00:00,남자,청년,수도권,행복,한식,일식,중식,17.5,1.6,...,1,0,0,1,0,0,1,05-16,14,6
1,2021-05-16 15:00:00,여자,중장년,서울,분노,한식,중식,일식,18.5,1.4,...,0,0,1,0,1,0,0,05-16,15,6
2,2021-05-16 15:00:00,남자,청년,서울,중립,일식,양식,한식,18.5,1.4,...,0,0,1,1,0,0,0,05-16,15,6
3,2021-05-16 15:00:00,여자,청년,서울,행복,일식,한식,분식,18.5,1.4,...,1,0,0,0,0,0,0,05-16,15,6
4,2021-05-16 16:00:00,남자,청년,경상도,행복,양식,일식,분식,20.6,2.6,...,1,0,0,1,0,0,0,05-16,16,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2021-05-24 16:00:00,남자,노년,경상도,행복,한식,분식,중식,23.8,0.0,...,0,0,0,0,0,0,1,05-24,16,0
417,2021-05-24 16:00:00,남자,노년,수도권,행복,한식,일식,중식,22.1,0.0,...,0,0,1,0,0,1,0,05-24,16,0
418,2021-05-24 16:00:00,여자,노년,전라도,행복,한식,중식,일식,22.7,0.0,...,0,0,0,1,1,0,0,05-24,16,0
419,2021-05-24 16:00:00,남자,노년,전라도,우울,분식,양식,일식,22.7,0.0,...,0,0,1,0,0,0,0,05-24,16,0


#### 요일을 한글로 변경

In [65]:
weekday = {0:'월', 1:'화', 2:'수', 3:'목', 4:'금', 5:'토', 6:'일'}

final_data.replace({'요일':weekday}, inplace=True)

In [66]:
final_data

,시간대,성별,연령대,지역,기분,1순위,2순위,3순위,기온,강수량,...,초밥,우동,메밀소바,돈가스,볶음밥,짜장면,짬뽕,날짜,시간,요일
0,2021-05-16 14:00:00,남자,청년,수도권,행복,한식,일식,중식,17.5,1.6,...,1,0,0,1,0,0,1,05-16,14,일
1,2021-05-16 15:00:00,여자,중장년,서울,분노,한식,중식,일식,18.5,1.4,...,0,0,1,0,1,0,0,05-16,15,일
2,2021-05-16 15:00:00,남자,청년,서울,중립,일식,양식,한식,18.5,1.4,...,0,0,1,1,0,0,0,05-16,15,일
3,2021-05-16 15:00:00,여자,청년,서울,행복,일식,한식,분식,18.5,1.4,...,1,0,0,0,0,0,0,05-16,15,일
4,2021-05-16 16:00:00,남자,청년,경상도,행복,양식,일식,분식,20.6,2.6,...,1,0,0,1,0,0,0,05-16,16,일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2021-05-24 16:00:00,남자,노년,경상도,행복,한식,분식,중식,23.8,0.0,...,0,0,0,0,0,0,1,05-24,16,월
417,2021-05-24 16:00:00,남자,노년,수도권,행복,한식,일식,중식,22.1,0.0,...,0,0,1,0,0,1,0,05-24,16,월
418,2021-05-24 16:00:00,여자,노년,전라도,행복,한식,중식,일식,22.7,0.0,...,0,0,0,1,1,0,0,05-24,16,월
419,2021-05-24 16:00:00,남자,노년,전라도,우울,분식,양식,일식,22.7,0.0,...,0,0,1,0,0,0,0,05-24,16,월


In [72]:
## 시간컬럼 타입 변경 object->int
final_data["시간"]=final_data["시간"].astype("int")

In [73]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   시간대     421 non-null    datetime64[ns]
 1   성별      421 non-null    object        
 2   연령대     421 non-null    object        
 3   지역      421 non-null    object        
 4   기분      421 non-null    object        
 5   1순위     421 non-null    object        
 6   2순위     421 non-null    object        
 7   3순위     421 non-null    object        
 8   기온      421 non-null    float64       
 9   강수량     421 non-null    float64       
 10  습도      421 non-null    float64       
 11  기압      421 non-null    float64       
 12  전운량     421 non-null    int64         
 13  불고기     421 non-null    int64         
 14  냉면      421 non-null    int64         
 15  칼국수     421 non-null    int64         
 16  된장찌개    421 non-null    int64         
 17  비빔밥     421 non-null    int64         
 18  김치찌개    42

### 휴일 컬럼 생성

In [67]:
# 평일 / 주말 구분 컬럼 생성

for i in range(len(final_data)):
    if final_data.loc[i, "요일"] == "일":
        final_data.loc[i, "휴일"] = "주말"
    else:
        final_data.loc[i, "휴일"] = "평일"

### 비 컬럼 생성

In [68]:
# 비 / 비안옴 구분 컬럼 생성

for i in range(len(final_data)):
    if final_data.loc[i, "강수량"] == 0:
        final_data.loc[i, "비"] = "비안옴"
    else:
        final_data.loc[i, "비"] ="비옴"

### 전운량 컬럼 생성

In [69]:
# 전운량 구분 컬럼 생성

final_data.loc[(final_data['전운량']>=0) & (final_data['전운량']<=5), '구름'] = "맑음"
final_data.loc[(final_data['전운량']>=6) & (final_data['전운량']<=8), '구름'] = "구름많음"
final_data.loc[(final_data['전운량']>=9) & (final_data['전운량']<=10), '구름'] = "흐림"

### 식사시간 컬럼 생성

In [74]:
# 식사시간 구분 컬럼 생성

for i in range(len(final_data)):
    if final_data.loc[i, "시간"]>=16 and final_data.loc[i, "시간"]<=23:
        final_data.loc[i, "식사시간"] = "저녁"
    elif final_data.loc[i, "시간"]>=11 and final_data.loc[i, "시간"]<16:
        final_data.loc[i, "식사시간"] = '점심'
    else:
        final_data.loc[i, "식사시간"] = "아침"

In [75]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   시간대     421 non-null    datetime64[ns]
 1   성별      421 non-null    object        
 2   연령대     421 non-null    object        
 3   지역      421 non-null    object        
 4   기분      421 non-null    object        
 5   1순위     421 non-null    object        
 6   2순위     421 non-null    object        
 7   3순위     421 non-null    object        
 8   기온      421 non-null    float64       
 9   강수량     421 non-null    float64       
 10  습도      421 non-null    float64       
 11  기압      421 non-null    float64       
 12  전운량     421 non-null    int64         
 13  불고기     421 non-null    int64         
 14  냉면      421 non-null    int64         
 15  칼국수     421 non-null    int64         
 16  된장찌개    421 non-null    int64         
 17  비빔밥     421 non-null    int64         
 18  김치찌개    42

### 최종 dataset 저장

In [76]:
final_data.to_csv('data/final_data.csv',index=False)